In [1]:
import os
import seaborn as sns
path = '/Users/connormcdonald/Desktop/Masters/MIT807/Gartner Repository/Analysis/Figures'
import sys
sys.path.insert(1, '/Users/connormcdonald/Desktop/Masters/MIT807/Gartner Repository/Data Collection')
from configs import *
import numpy as np
import statsmodels.formula.api as smf
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from mpl_toolkits.axisartist.axislines import SubplotZero
from pylab import text
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "lualatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

from sqlalchemy import create_engine

engine = create_engine('mysql+mysqlconnector://'+user+':'+passwd+'@'+ip+':3306/'+schema1)

In [2]:
df = pd.read_csv('/Users/connormcdonald/Desktop/Masters/MIT807/Data/twitterMAU.csv')
df['period']  = pd.to_datetime(df['period'])

In [3]:
df1 = df.iloc[:38,:]

In [4]:
import matplotlib.dates as mdates


degree = 3
y = np.array(df1['mau'].to_list())
X = np.array(mdates.date2num(df1['period']))
date_index = []
idx = 0
for i in X:
    date_index.append(idx)
    idx += 1

z = np.polyfit(date_index, y, degree)
f = np.poly1d(z)

df_trend = pd.DataFrame(columns=['X', 'y','d'])
df_trend['X'] = date_index
df_trend['y'] = y
df_trend['d'] = X


z = np.polyfit(df_trend.d, df_trend.y, degree)
model = np.poly1d(z)
results = smf.ols(formula='y ~ model(X)', data=df_trend).fit()

# print(results.summary().as_latex())

In [5]:
X = mdates.date2num(df['period'])

In [6]:
x_fit = mdates.date2num(df['period'])
y_fit = [model(_x) for _x in x_fit]

In [7]:
def dformat(d):
    return d.strftime('%Y-%m')

def yformat(d):
    return d.strftime('%Y')

In [8]:
df['y_fit'] = y_fit
df['combined_mau'] = np.where(df["mau"].isnull(), df["y_fit"], df["mau"] )*1000000
df['period_formatted'] = df['period'].apply(dformat)

In [9]:
stmt = '''SELECT DATE_FORMAT(date, \'%Y-%m\') as date, 
COUNT(1) AS tweets,
SUM(like_count) as likes,
SUM(quote_count) as quotes,
SUM(reply_count) as comments,
SUM(retweet_count) as retweets
FROM social.five_g_only 
group by DATE_FORMAT(date, \'%Y-%m\') 
ORDER BY DATE_FORMAT(date, \'%Y-%m\') ASC'''
df2 = pd.read_sql(stmt, con=engine)

df3 = pd.merge(df, df2, left_on='period_formatted', right_on='date', how='inner')

df3['tweets_p_user'] = df3['tweets']/df3['combined_mau']

df3['likes_p_user'] = df3['likes']/df3['combined_mau']
df3['comments_p_user'] = df3['comments']/df3['combined_mau']
df3['quotes_p_user'] = df3['quotes']/df3['combined_mau']
df3['retweets_p_user'] = df3['retweets']/df3['combined_mau']

df3['total_engagement'] = df3['likes_p_user'] + df3['comments_p_user'] + df3['quotes_p_user'] # removed retweets_p_user

In [10]:
correlation_df = df3
correlation_df['date']  = pd.to_datetime(correlation_df['date'])

correlation_pre = correlation_df[correlation_df.period_formatted < '2020-03-01']
correlation_post = correlation_df[correlation_df.period_formatted >= '2020-03-01']


column_1 = correlation_df["tweets_p_user"]
column_2 = correlation_df["total_engagement"]
correlation = column_1.corr(column_2)
print(f"total: {correlation}")


column_1 = correlation_pre["tweets_p_user"]
column_2 = correlation_pre["total_engagement"]
correlation = column_1.corr(column_2)
print(f"pre: {correlation}")


column_1 = correlation_post["tweets_p_user"]
column_2 = correlation_post["total_engagement"]
correlation = column_1.corr(column_2)
print(f"post: {correlation}")

total: 0.9097259909114175
pre: 0.9097259909114175
post: nan


In [11]:
from scipy.stats.stats import pearsonr
corr = []
c = []
p = []
for i in range(2, len(correlation_df.index)):
    temp_df = correlation_df.iloc[:i,:]
    tweets = temp_df.tweets_p_user
    engagement = temp_df.total_engagement
    correlation = tweets.corr(engagement)
    corr.append(correlation)
    x = pearsonr(engagement, tweets)
    c.append(x[0])
    p.append(x[1])

/var/folders/c7/dcyx4ss130ldp4lp9ph0v16c0000gn/T/ipykernel_1676/2356826354.py:1: DeprecationWarning: Please use `pearsonr` from the `scipy.stats` namespace, the `scipy.stats.stats` namespace is deprecated.
  from scipy.stats.stats import pearsonr


In [12]:
#corr plot
line = pd.to_datetime('2017-06-01')
line2 = pd.to_datetime('2018-09-01')
# line3 = pd.to_datetime('2017-12-01')
# line4 = pd.to_datetime('2014-06-01')
ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.rcParams["figure.figsize"] = [8, 4]
plt.rcParams.update({'font.size': 12})
plt.rcParams['figure.dpi'] = 300
plt.xlabel('Time')
plt.ylabel('Correlation')
plt.plot(correlation_df.loc[2:].date, corr, c = 'black', linewidth = 1)
plt.axvline(x=line, c = '#E31B23', linewidth = 1, linestyle ="--", label= 'POI-A')
plt.axvline(x=line2, c = '#999999', linewidth = 1, linestyle ="--", label = "POI-B")
# plt.axvline(x=line3, c = '#999999', linewidth = 1, linestyle ="--", label= 'POI-C')
# plt.axvline(x=line2, c = '#FF9A00', linewidth = 1, linestyle ="--", label= 'POI-D')
plt.legend(loc='upper left')
plt.savefig(os.path.join(path, '5g_correleation.pdf'), format='pdf',bbox_inches='tight',pad_inches = 0)
plt.close()


In [13]:
degree = 3
y = np.array(df3['tweets_p_user'].to_list())
X = np.array(mdates.date2num(df3['date']))
date_index = []
idx = 0
for i in X:
    date_index.append(idx)
    idx += 1

z = np.polyfit(date_index, y, degree)
f = np.poly1d(z)

df_trend = pd.DataFrame(columns=['X', 'y','d'])
df_trend['X'] = date_index
df_trend['y'] = y
df_trend['d'] = X


z = np.polyfit(df_trend.d, df_trend.y, degree)
model = np.poly1d(z)
results = smf.ols(formula='y ~ model(X)', data=df_trend).fit()


X = mdates.date2num(df3['date'])

x_fit = mdates.date2num(df3['date'])
y_fit = [model(_x) for _x in x_fit]

df3['y_fit_2'] = y_fit

In [14]:
print(results.summary().as_latex())

\begin{center}
\begin{tabular}{lclc}
\toprule
\textbf{Dep. Variable:}    &        y         & \textbf{  R-squared:         } &     0.698   \\
\textbf{Model:}            &       OLS        & \textbf{  Adj. R-squared:    } &     0.689   \\
\textbf{Method:}           &  Least Squares   & \textbf{  F-statistic:       } &     78.60   \\
\textbf{Date:}             & Tue, 28 Mar 2023 & \textbf{  Prob (F-statistic):} &  2.32e-10   \\
\textbf{Time:}             &     09:33:18     & \textbf{  Log-Likelihood:    } &    304.86   \\
\textbf{No. Observations:} &          36      & \textbf{  AIC:               } &    -605.7   \\
\textbf{Df Residuals:}     &          34      & \textbf{  BIC:               } &    -602.6   \\
\textbf{Df Model:}         &           1      & \textbf{                     } &             \\
\textbf{Covariance Type:}  &    nonrobust     & \textbf{                     } &             \\
\bottomrule
\end{tabular}
\begin{tabular}{lcccccc}
                   & \textbf{coef} & \t

In [15]:
df3.head()

,period,mau,y_fit,combined_mau,period_formatted,date,tweets,likes,quotes,comments,retweets,tweets_p_user,likes_p_user,comments_p_user,quotes_p_user,retweets_p_user,total_engagement,y_fit_2
0,2010-03-31,30.0,5.282818,30000000.0,2010-03,2010-03-01,3012,36.0,0.0,0.0,253.0,0.000100,1.200000e-06,0.000000e+00,0.000000e+00,0.000008,1.200000e-06,0.000098
1,2010-06-30,40.0,25.106319,40000000.0,2010-06,2010-06-01,4502,34.0,0.0,0.0,1578.0,0.000113,8.500000e-07,0.000000e+00,0.000000e+00,0.000039,8.500000e-07,0.000095
2,2010-09-30,49.0,44.504028,49000000.0,2010-09,2010-09-01,3170,31.0,0.0,2.0,888.0,0.000065,6.326531e-07,4.081633e-08,0.000000e+00,0.000018,6.734694e-07,0.000093
3,2010-12-31,54.0,63.256920,54000000.0,2010-12,2010-12-01,4814,86.0,0.0,1.0,2991.0,0.000089,1.592593e-06,1.851852e-08,0.000000e+00,0.000055,1.611111e-06,0.000091
4,2011-03-31,68.0,80.980447,68000000.0,2011-03,2011-03-01,8282,85.0,1.0,2.0,12041.0,0.000122,1.250000e-06,2.941176e-08,1.470588e-08,0.000177,1.294118e-06,0.000089


In [16]:


curr_max_freq = 0
curr_max_eng = 0
for i in range(len(df3.index)-1):
    a = df3.tweets_p_user[i]
    b = df3.tweets_p_user[i+1]

    c = df3.total_engagement[i]
    d = df3.total_engagement[i+1]


    if a > b and i !=0 and c > d and a > curr_max_freq and c > curr_max_eng and c > a:
        print(df3.period_formatted[i])
        curr_max_freq = df3.tweets_p_user[i]
        curr_max_eng = df3.total_engagement[i]

2017-06
2018-09


In [17]:
#when engagement overtook tweets

for i in range(len(df3.index)):
    a = df3.tweets_p_user[i]
    b = df3.total_engagement[i]

    if a < b: 
        print(df3.period_formatted[i])

2017-06
2017-09
2017-12
2018-03
2018-06
2018-09
2018-12


In [18]:
#ML plot
df3['date']  = pd.to_datetime(df3['date'])
ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.rcParams["figure.figsize"] = [6,6]
plt.rcParams.update({'font.size': 12})
plt.rcParams['figure.dpi'] = 300
plt.xlabel('Time')
plt.ylabel('Monthly Tweets Per Active User')
plt.plot(df3['date'], df3['tweets_p_user'], c = '#339898', linewidth = 1, label = 'Tweets')
plt.plot(df3['date'], df3['y_fit_2'], c = '#E31B23', linewidth = 1, linestyle ="--", label= 'Polynomial Trend Line ($R^2$ = 0.698)')
plt.legend(loc='upper left')
plt.savefig(os.path.join(path, '5g_normalised_tweets.pdf'), format='pdf',bbox_inches='tight',pad_inches = 0)
plt.close()



In [20]:
ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.rcParams["figure.figsize"] = [6,6]
plt.rcParams.update({'font.size': 12})
plt.rcParams['figure.dpi'] = 300


#define colors to use in chart
color_map = ['#339898', '#E31B23', '#999999', '#FF9A00']

#create area chart
plt.stackplot(df3['date'], df3['likes_p_user'], df3['comments_p_user'], df3['quotes_p_user'],df3['retweets_p_user'],
              labels=['Likes', 'Comments', 'Quotes', 'Retweets'],
              colors=color_map,
              alpha=0.6)
plt.plot(df3['date'], df3['tweets_p_user'], c = 'black', linewidth = 1,label = 'Tweets')

#add legend
plt.legend(loc='upper left')

#add axis labels
plt.xlabel('Time')
plt.ylabel('Monthly Metrics Per Active User')
plt.savefig(os.path.join(path, '5g_normalised_area.pdf'), format='pdf',bbox_inches='tight',pad_inches = 0)
plt.close()



In [17]:
ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.rcParams["figure.figsize"] = [8, 4]
plt.rcParams.update({'font.size': 12})
plt.rcParams['figure.dpi'] = 300


#define colors to use in chart
color_map = ['#339898', '#E31B23', '#999999']

#create area chart
plt.stackplot(df3['date'], df3['likes_p_user'], df3['comments_p_user'], df3['quotes_p_user'],
              labels=['Likes', 'Comments', 'Quotes'],
              colors=color_map,
              alpha=0.6)
plt.plot(df3['date'], df3['tweets_p_user'], c = 'black', linewidth = 1,label = 'Tweets')

#add legend
plt.legend(loc='upper left')

#add axis labels
plt.xlabel('Time')
plt.ylabel('Monthly Metrics Per Active User')
plt.show()
plt.savefig(os.path.join(path, '5g_normalised_area2.pdf'), format='pdf',bbox_inches='tight',pad_inches = 0)
plt.close()

<ipython-input-17-f7b055bcb435>:25: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [19]:
#5g plot
line = pd.to_datetime('2017-06-01')
line2 = pd.to_datetime('2018-09-01')
# line3 = pd.to_datetime('2014-06-01')
# line4 = pd.to_datetime('2015-03-01')

ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.rcParams["figure.figsize"] = [6,6]
plt.rcParams.update({'font.size': 12})
plt.rcParams['figure.dpi'] = 300
plt.xlabel('Time')
plt.ylabel('Monthly Tweets Per Active User')
plt.plot(df3['date'], df3['tweets_p_user'], c = 'black', linewidth = 1, label = 'Tweets')
plt.axvline(x=line, c = '#E31B23', linewidth = 1, linestyle ="--", label= 'POI-A')
plt.axvline(x=line2, c = '#999999', linewidth = 1, linestyle ="--", label = "POI-B")
plt.legend(loc='upper left')
plt.savefig(os.path.join(path, '5g_shift.pdf'), format='pdf',bbox_inches='tight',pad_inches = 0)
plt.close()



In [28]:
df = pd.read_csv('/Users/connormcdonald/Desktop/Masters/MIT807/Data/academic_results.csv')

In [29]:
year = ['2010-12-31', '2011-12-31', '2012-12-31', '2013-12-31', '2014-12-31', '2015-12-31', '2016-12-31', '2017-12-31', '2018-12-31', '2019-12-31', '2020-12-31', '2021-12-31']
papers = [585, 881, 921, 874, 1365, 2024, 3225, 4740, 6857, 9050, 9579, 12018]
df['papers'] = papers

In [30]:
df['scopus_norm'] = df['papers']/df['scopus_results']

In [31]:
df

,period,scholar_results,scopus_results,papers,scopus_norm
0,2010,4510000,2478126,585,0.000236
1,2011,4590000,2638921,881,0.000334
2,2012,4670000,2775816,921,0.000332
3,2013,4360000,2901649,874,0.000301
4,2014,4640000,2942515,1365,0.000464
5,2015,4710000,2953613,2024,0.000685
6,2016,3040000,3055033,3225,0.001056
7,2017,3450000,3151892,4740,0.001504
8,2018,3960000,3283566,6857,0.002088
9,2019,3500000,3437602,9050,0.002633


In [32]:
import matplotlib.dates as mdates
degree = 3
y = np.array(df['scopus_norm'].to_list())
X = np.array(mdates.date2num(df['period']))
date_index = []
idx = 0
for i in X:
    date_index.append(idx)
    idx += 1

z = np.polyfit(date_index, y, degree)
f = np.poly1d(z)

df_trend = pd.DataFrame(columns=['X', 'y','d'])
df_trend['X'] = date_index
df_trend['y'] = y
df_trend['d'] = X


z = np.polyfit(df_trend.d, df_trend.y, degree)
model = np.poly1d(z)
results = smf.ols(formula='y ~ model(X)', data=df_trend).fit()


X = mdates.date2num(df['period'])

x_fit = mdates.date2num(df['period'])
y_fit = [model(_x) for _x in x_fit]

df['y_fit_2'] = y_fit

In [33]:
print(results.summary().as_latex())

\begin{center}
\begin{tabular}{lclc}
\toprule
\textbf{Dep. Variable:}    &        y         & \textbf{  R-squared:         } &     0.829   \\
\textbf{Model:}            &       OLS        & \textbf{  Adj. R-squared:    } &     0.829   \\
\textbf{Method:}           &  Least Squares   & \textbf{  F-statistic:       } &       nan   \\
\textbf{Date:}             & Tue, 28 Mar 2023 & \textbf{  Prob (F-statistic):} &      nan    \\
\textbf{Time:}             &     08:25:27     & \textbf{  Log-Likelihood:    } &    76.113   \\
\textbf{No. Observations:} &          12      & \textbf{  AIC:               } &    -150.2   \\
\textbf{Df Residuals:}     &          11      & \textbf{  BIC:               } &    -149.7   \\
\textbf{Df Model:}         &           0      & \textbf{                     } &             \\
\textbf{Covariance Type:}  &    nonrobust     & \textbf{                     } &             \\
\bottomrule
\end{tabular}
\begin{tabular}{lcccccc}
                   & \textbf{coef} & \t

/Users/connormcdonald/opt/anaconda3/lib/python3.8/site-packages/scipy/stats/_stats_py.py:1772: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=12
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [36]:
#ML plot
# df['period']  = pd.to_datetime(df['period'])
ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.rcParams["figure.figsize"] = [6,6]
plt.rcParams.update({'font.size': 12})
plt.rcParams['figure.dpi'] = 300
plt.xlabel('Time')
plt.ylabel('Proportion of 5G Publications')
plt.plot(df['period'], df['scopus_norm'], c = '#339898', linewidth = 1, label ='Publications')
plt.plot(df['period'], df['y_fit_2'], c = '#E31B23', linewidth = 1, linestyle ="--", label= 'Polynomial Trend Line ($R^2$ = 0.829)')
plt.legend(loc='upper left')
plt.savefig(os.path.join(path, '5g_normalised_pubs.pdf'), format='pdf',bbox_inches='tight',pad_inches = 0)
plt.close()

In [35]:
stmt = 'SELECT CHAR_LENGTH (text) as char_count, YEAR(date) as year FROM social.five_g_only'

df = pd.read_sql(stmt, con=engine)

KeyboardInterrupt: 

In [9]:
ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax = sns.violinplot(x="year", y="char_count", data=df, linewidth=1)
plt.rcParams["figure.figsize"] = [8, 4]
plt.rcParams.update({'font.size': 12})
plt.rcParams['figure.dpi'] = 300
plt.xlabel('Year')
plt.ylabel('Character Count Per Tweet')
plt.savefig(os.path.join(path, '5g_char_count.pdf'), format='pdf',bbox_inches='tight',pad_inches = 0)
plt.close()

In [2]:
#Innovation Trigger

stmt = '''
SELECT A.text
FROM social.five_g_only A
WHERE A.date >= \'2017-05-01\' and A.date <= \'2017-06-30\'
'''

df = pd.read_sql(stmt, con=engine)

In [3]:
# Defining dictionary containing all emojis with their meanings.
emojis = {':)': 'smile', ':-)': 'smile', ';d': 'wink', ':-E': 'vampire', ':(': 'sad', 
          ':-(': 'sad', ':-<': 'sad', ':P': 'raspberry', ':O': 'surprised',
          ':-@': 'shocked', ':@': 'shocked',':-$': 'confused', ':\\': 'annoyed', 
          ':#': 'mute', ':X': 'mute', ':^)': 'smile', ':-&': 'confused', '$_$': 'greedy',
          '@@': 'eyeroll', ':-!': 'confused', ':-D': 'smile', ':-0': 'yell', 'O.o': 'confused',
          '<(-_-)>': 'robot', 'd[-_-]b': 'dj', ":'-)": 'sadsmile', ';)': 'wink', 
          ';-)': 'wink', 'O:-)': 'angel','O*-)': 'angel','(:-D': 'gossip', '=^.^=': 'cat'}
          
## Defining set containing all stopwords in english.
stopwordlist = ['a', 'about', 'above', 'after', 'again', 'ain', 'all', 'am', 'an',
             'and','any','are', 'as', 'at', 'be', 'because', 'been', 'before',
             'being', 'below', 'between','both', 'by', 'can', 'd', 'did', 'do',
             'does', 'doing', 'down', 'during', 'each','few', 'for', 'from', 
             'further', 'had', 'has', 'have', 'having', 'he', 'her', 'here',
             'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in',
             'into','is', 'it', 'its', 'itself', 'just', 'll', 'm', 'ma',
             'me', 'more', 'most','my', 'myself', 'now', 'o', 'of', 'on', 'once',
             'only', 'or', 'other', 'our', 'ours','ourselves', 'out', 'own', 're',
             's', 'same', 'she', "shes", 'should', "shouldve",'so', 'some', 'such',
             't', 'than', 'that', "thatll", 'the', 'their', 'theirs', 'them',
             'themselves', 'then', 'there', 'these', 'they', 'this', 'those', 
             'through', 'to', 'too','under', 'until', 'up', 've', 'very', 'was',
             'we', 'were', 'what', 'when', 'where','which','while', 'who', 'whom',
             'why', 'will', 'with', 'won', 'y', 'you', "youd","youll", "youre",
             "youve", 'your', 'yours', 'yourself', 'yourselves']

# nltk
from nltk.stem import WordNetLemmatizer
import re

def preprocess(textdata):
    processedText = []
    
    # Create Lemmatizer and Stemmer.
    wordLemm = WordNetLemmatizer()
    
    # Defining regex patterns.
    urlPattern        = r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*)"
    userPattern       = '@[^\s]+'
    alphaPattern      = "[^a-zA-Z0-9]"
    sequencePattern   = r"(.)\1\1+"
    seqReplacePattern = r"\1\1"
    
    for tweet in textdata:
        tweet = tweet.lower()
        
        # Replace all URls with 'URL'
        tweet = re.sub(urlPattern,' URL',tweet)
        # Replace all emojis.
        for emoji in emojis.keys():
            tweet = tweet.replace(emoji, "EMOJI" + emojis[emoji])        
        # Replace @USERNAME to 'USER'.
        tweet = re.sub(userPattern,' USER', tweet)        
        # Replace all non alphabets.
        tweet = re.sub(alphaPattern, " ", tweet)
        # Replace 3 or more consecutive letters by 2 letter.
        tweet = re.sub(sequencePattern, seqReplacePattern, tweet)

        tweetwords = ''
        for word in tweet.split():
            # Checking if the word is a stopword.
            #if word not in stopwordlist:
            if len(word)>1:
                # Lemmatizing the word.
                word = wordLemm.lemmatize(word)
                tweetwords += (word+' ')
            
        processedText.append(tweetwords)
        
    return processedText

In [4]:
import time
text = df['text']
t = time.time()
processedtext = preprocess(text)
print(f'Text Preprocessing complete.')
print(f'Time Taken: {round(time.time()-t)} seconds')

Text Preprocessing complete.
Time Taken: 8 seconds


In [5]:
from wordcloud import WordCloud
from nltk.corpus import stopwords

#Total Wordcloud
allwords = ' '.join([tweets for tweets in processedtext]) 
stop_words = stopwords.words('english')
stop_words.append('5g')

In [7]:
plt.rcParams['figure.dpi'] = 300
# plt.rcParams["figure.figsize"] = [8, 4]
wordCloud = WordCloud(
    font_path='/Library/Fonts/cmunrm.ttf',
    width=800, 
    height=800,
    background_color="white",
    max_font_size = 200,
    min_font_size = 15,
    collocation_threshold = 50,
    stopwords=stop_words,
    random_state=1,
    colormap="gist_heat").generate(allwords)
plt.imshow(wordCloud, interpolation = 'bilinear')
plt.axis('off')
plt.savefig(os.path.join(path, '5g_innovation_trigger.pdf'), format='pdf',bbox_inches='tight',pad_inches = 0)
plt.close()

In [8]:
#first peak

stmt = '''
SELECT A.text
FROM social.five_g_only A
WHERE A.date >= \'2018-08-01\' and A.date <= \'2018-09-30\'
'''

df = pd.read_sql(stmt, con=engine)

In [9]:
text = df['text']
t = time.time()
processedtext = preprocess(text)
print(f'Text Preprocessing complete.')
print(f'Time Taken: {round(time.time()-t)} seconds')

Text Preprocessing complete.
Time Taken: 16 seconds


In [10]:
allwords = ' '.join([tweets for tweets in processedtext])

plt.rcParams['figure.dpi'] = 300
wordCloud = WordCloud(
    font_path='/Library/Fonts/cmunrm.ttf',
    width=800, 
    height=800,
    background_color="white",
    max_font_size = 200,
    min_font_size = 15,
    collocation_threshold = 50,
    stopwords=stop_words,
    random_state=1,
    colormap="gist_heat").generate(allwords)
plt.imshow(wordCloud, interpolation = 'bilinear')
plt.axis('off')
plt.savefig(os.path.join(path, '5g_peak_1.pdf'), format='pdf',bbox_inches='tight',pad_inches = 0)
plt.close()